# Custom Formats and Errors

## Using pd.to_datetime with custom date formats

- Although pd.to_datetime can typically convert datetimes without error, it will need some help with rarer date formats.
- Format info can be passed along to help convert data properly.

In [1]:
import pandas as pd
demo = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT_jChgNsQbHbg4TGepzIqk8XC9DTIKmyyxb1upo5cfZCgbfIUQc2ZC0YMzuU5uApP140Ob49KBjdqh/pub?gid=1198589591&single=true&output=csv', usecols=[0,1])
demo.info()
demo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15341 entries, 0 to 15340
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         15341 non-null  int64  
 1   cloud_cover  14938 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 239.8 KB


date  cloud_cover
0      19790101          2.0
1      19790102          6.0
2      19790103          5.0
3      19790104          8.0
4      19790105          6.0
...         ...          ...
15336  20201227          1.0
15337  20201228          7.0
15338  20201229          7.0
15339  20201230          6.0
15340  20201231          7.0

[15341 rows x 2 columns]

- Let's attempt to convert this data using pd.to_datetime without any arguments

In [2]:
pd.to_datetime(demo['date'])

0       1970-01-01 00:00:00.019790101
1       1970-01-01 00:00:00.019790102
2       1970-01-01 00:00:00.019790103
3       1970-01-01 00:00:00.019790104
4       1970-01-01 00:00:00.019790105
                     ...             
15336   1970-01-01 00:00:00.020201227
15337   1970-01-01 00:00:00.020201228
15338   1970-01-01 00:00:00.020201229
15339   1970-01-01 00:00:00.020201230
15340   1970-01-01 00:00:00.020201231
Name: date, Length: 15341, dtype: datetime64[ns]

- We can see above it did not convert properly. Pandas attempted to convert as though the data was given in Unix times, which are a way of expressing date and times with just numbers. The earliest date available to be expressed this way is Jan 1, 1970.
- This is a case where additional instructions must be passed when calling pd.to_datetime

In [3]:
# Display a random sample to asses the original data
demo['date'].sample(n=10).sort_index()

2745     19860708
4415     19910202
5421     19931104
8436     20020205
9635     20050519
9683     20050706
12903    20140430
13091    20141104
14619    20190110
15238    20200920
Name: date, dtype: int64

- We have a 4-digit year, 2-digit month, and 2-digit day.
- Once we have our original format, we can create a format string to use for conversion.
- Our format string will be '%Y%m%d'

In [4]:
pd.to_datetime(demo['date'], format='%Y%m%d')

0       1979-01-01
1       1979-01-02
2       1979-01-03
3       1979-01-04
4       1979-01-05
           ...    
15336   2020-12-27
15337   2020-12-28
15338   2020-12-29
15339   2020-12-30
15340   2020-12-31
Name: date, Length: 15341, dtype: datetime64[ns]

## Handling pd.to_datetime errors

In [5]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4lekzpYpo0pA9h1d3KYObIb3lQtAqz289c7jpwckioXvxm4xykz6ZSJpnDwjKTxJ4iqpG0seNJdSZ/pub?gidass=>=312387697&single=true&output=csv"
demo = pd.read_csv(url)
demo.head(3)

CrimeDateTime          Description   District  Latitude  Longitude
0  2022/07/09 09:30:00+00  ROBBERY - RESIDENCE  NORTHEAST   39.3223   -76.5467
1  2022/07/09 16:00:00+00       COMMON ASSAULT   SOUTHERN   39.2821   -76.6355
2  2022/07/09 00:34:28+00             SHOOTING  SOUTHWEST   39.2884   -76.6569

In [6]:
# Using this code will generate an error
# pd.to_datetime(demo['CrimeDateTime'])

- The error message is quite large, but this will give us some insight into what is causing the problem

In [7]:
try:
    display(pd.to_datetime(demo['CrimeDateTime']))
except Exception as e:
    print(e)

Out of bounds nanosecond timestamp: 1202-05-22 10:56:02 present at position 522867


- Sometimes there may be 'bad' values that are not compatible with the other dates in the column.
- In these cases we can leverage the errors arg of pd.to_datetime
- According to the docstring for pd.to_datetime
- Errors:
    - if 'raise' then invalid parsing will raise an exception
    - if 'coerce' then invalid parsin will be set as NaT
    - if 'ignore' then invalid parsing will return the input

In [8]:
# Using errors='coerce' will replace incompatible vaalues with NaT (not a time)
demo['CrimeDateTime'] = pd.to_datetime(demo['CrimeDateTime'], errors='coerce')
demo['CrimeDateTime'].isna().sum()

1

- Remember to drop or impute any null values fro time. In this case we simply drop the one value.

In [9]:
demo = demo.dropna(subset=['CrimeDateTime'])
demo.head()

CrimeDateTime          Description   District  Latitude  \
0 2022-07-09 09:30:00+00:00  ROBBERY - RESIDENCE  NORTHEAST   39.3223   
1 2022-07-09 16:00:00+00:00       COMMON ASSAULT   SOUTHERN   39.2821   
2 2022-07-09 00:34:28+00:00             SHOOTING  SOUTHWEST   39.2884   
3 2022-07-09 00:34:28+00:00             SHOOTING  SOUTHWEST   39.2884   
4 2022-07-09 18:00:00+00:00       COMMON ASSAULT  NORTHEAST   39.3188   

   Longitude  
0   -76.5467  
1   -76.6355  
2   -76.6569  
3   -76.6569  
4   -76.5872

- Now that we have dropped the data point causing the error we can try again

In [10]:
pd.to_datetime(demo['CrimeDateTime'])

0        2022-07-09 09:30:00+00:00
1        2022-07-09 16:00:00+00:00
2        2022-07-09 00:34:28+00:00
3        2022-07-09 00:34:28+00:00
4        2022-07-09 18:00:00+00:00
                    ...           
527812   2013-07-31 17:45:00+00:00
527813   2013-07-31 14:15:00+00:00
527814   2013-07-31 21:00:00+00:00
527815   2013-07-31 17:00:00+00:00
527816   2013-07-31 11:40:00+00:00
Name: CrimeDateTime, Length: 527816, dtype: datetime64[ns, UTC]

## Summary

- If a unique format in the original dataset causes errors, we can pass a format string to help Pandas convert.
- If we receive an error due to one or a few datapoints, we can convert to NaT and drop.
- Be sure to check output along the way to ensure we are getting the desired result.